In [1]:
import sys
sys.path.append('../')

In [2]:
%load_ext autoreload
%autoreload 2

In [4]:
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv(), override=True)

---

In [49]:
import requests
import json

import os
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)

from group_data import df_pipe
from src.prompts.options import next_state_options
from src.prompts.system_prompt_tracks_v2 import sys_prompt
from src.prompts.user_prompt_tracks_v2 import (
    examples, 
    user, 
    parse_data_object, 
    build_user_message
)
from src.enums.states import States
from src.enums.tracks import track_mapping
import tiktoken

### Import Data + Initialize Encoder

In [50]:
data_path = '/Users/americanthinker/Downloads/antx_data/2024-05-14_all_tracks.csv'
data = df_pipe(data_path, fill_state=False, merge_tracks=True)
encoder = tiktoken.get_encoding('cl100k_base')

In [51]:
#parse out only the data rows where a state change is present
states = [{k:v} for k,v in data.items() if any(v['state'])]
len(states)

25

In [52]:
keys = list(data.keys())
len(keys)

564

In [53]:
minute_value = '12:26'
tracks = parse_data_object(data[minute_value])
current_state = [s for s in data[minute_value]['state'] if s][0]

In [54]:
user_message = build_user_prompt(examples, current_state, tracks, next_state_options[current_state]['Options'])
print(user_message)


As context you will be given a Current State and a series of radio transmissions broken out into separate tracks. 
The transmissions represent dialogue between stakeholders involved in maritime UAV test and evaluation trials.
  
Given the context perform the following two sequential tasks:
1. Decide if a change to the Current State is warranted given the transmissions. 
   - If no change is warranted simply return a json object as follows:
   - Output: {"current_state": <current state>}

2. Based on your decision from step 1, if a change in state is warranted then choose from among the next state options and return your output to include, current state, predicted state, and your reasoning for choosing the next predicted state.  Follow the output guidelines below:
    - Output: {"current_state": <current state>, "predicted_state": <predicted state>, "reason": <reason for predicted state>}


EXAMPLES
--------------------------
Use the following examples as your guide for predicting whet

In [55]:
len(encoder.encode(user_message + sys_prompt))

2624

In [557]:
# for x in range(1,5):
#     if x != 2:
#         track = [t for t in df[f'track{x}'].values.tolist() if t]
#         print(f'Track {x} length: {len(track)}')

In [12]:
# labeled_data = states.loc[:,'track1':'state']
# labeled_data_cleaned = []
# labeled_data_dicts = []
# for alist in labeled_data.values:
#     cleaned = [text for text in alist if text]
#     labeled_data_cleaned.append(cleaned)
# for alist in labeled_data_cleaned:
#     datum = {'text': ' '.join([t for t in alist[:-1]]), 'label': alist[-1]}
# #     labeled_data_dicts.append(datum)
# ordered_labels = [d['label'] for d in labeled_data_dicts]
# ordered_labels

### LLM Calls

In [13]:
def format_response(response: requests.models.Response):
    json_response = response.json()
    return json_response['choices'][0]['message']['content'].strip()

In [94]:
def chat_completion(user_prompt: str,
                    temperature: float=0.8,
                    max_tokens: int=4096,
                    stream: bool=False,
                    raw: bool=False
                    ) -> str | dict:
    url = os.environ['LEAPFROG_URL']
    api_key = os.environ['LEAPFROG_API_KEY']
    headers = {
    'Authorization': f'Bearer {api_key}',
    'Content-Type': 'application/json'
    }
    data = {
    "model": "vllm",
    "messages": [
        {
            "role": "system",
            "content": sys_prompt
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ],
    "stream": stream,
    "temperature": temperature,
    "max_tokens": max_tokens
}
    try:
        response = requests.post(url, headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            if raw:
                return response.json()
            else: return format_response(response)
        else:
            print('Response is not 200')
            return response
    except Exception as e:
        print(e)

In [581]:
response = chat_completion(current_state, max_tokens=250)
response

Trial Start


In [62]:
list(states[0].keys())[0]

'08:42'

In [ ]:
minute_value = '12:26'
tracks = parse_data_object(data[minute_value])
current_state = [s for s in data[minute_value]['state'] if s][0]

In [103]:
def test_harness(test_data: list[dict]) -> list[str]:
    current_state = 'Trial Start'
    responses = []
    for i, data_point in enumerate(test_data):
        minute_value = list(data_point.keys())[0]
        tracks = parse_data_object(data_point[minute_value])
        predicted_state = [s for s in data_point[minute_value]['state'] if s][0]
        user_message = build_user_message(examples, current_state, tracks, next_state_options[current_state]['Options'])
        responses.append({'response':chat_completion(user_message, max_tokens=250), 'label':predicted_state, 'minute': minute_value})
        current_state = predicted_state
    return responses

In [105]:
test = test_harness(states[1:])

In [126]:
miss_verbiage = [
    'To be advised, we got a helo flying low over the north corner of the field.',
    "Yeah. Yeah? Mistrial? Mistrial. OK. All call, all call. Trial completion, trial completion. So, yeah. It's passed. So there were, Kate filled me in, there were a few frames, maybe there was something in frame, but they didn't have a lawyer now. Go for Travis.",
    "So, this will be trial two, two, one. And if you're ready, I'm gonna give the call. All call, all call. Trial 2-2-1 has begun. All call, all call. Trial 2-2-1 has begun.",
    "the trial and we'll just All calls, trial end. Let's have red vehicle reset, and we will restart. Yeah, reset those bad boys.",
    "I'm like a Mistrial was requested trial and trial and resetting for next trial"
]

In [127]:
misses = 6
hits = 5

In [128]:
states[11]

{'12:27': {'track1': '',
  'track3': "OK. And that's what the economy software is on right now. Yeah. And so during this, the evolution of this event, we are gonna port this software back on the GARC. So in IBP 24.1, we put this on the GARC, but it was independent from the rest of this grouping of vehicles. So now the intention is to have it be, you know, same way that, you know, have it just be another vehicle that EpicSci is controlling. And then, are the, Not as the GARC so they have a separate low-level controller Which autopilot will call it that and that is developed by OPT, right? So yeah, the way MV ship with",
  'track4': "So 310 and 15 knots. I like to just enter it directly into the field. I'm a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm like a numbers guy. I'm lik

In [122]:
test

[{'response': '{"current_state": Trial Start}</s>',
  'label': 'Delay Start',
  'minute': '09:42'},
 {'response': '{"current_state": Delay Start}</s>',
  'label': 'Delay Start',
  'minute': '09:50'},
 {'response': '{"current_state": Delay Start, "predicted_state": Delay End, "reason": "Radio transmission from the Test Director indicates that the issue with the network has been resolved and the trial can proceed."}</s>',
  'label': 'Delay End',
  'minute': '10:23'},
 {'response': '{"current_state": Delay End}</s>',
  'label': 'Delay End',
  'minute': '10:27'},
 {'response': '{"current_state": Delay End, "predicted_state": Trial Start, "reason": "Radio transmission from the Test Director indicates that a new trial (211) has just commenced."}',
  'label': 'Trial Start',
  'minute': '11:48'},
 {'response': '{"current_state": Trial Start}</s>',
  'label': 'Delay Start',
  'minute': '11:55'},
 {'response': '{"current_state": Delay Start, "predicted_state": Delay End, "reason": "Radio transmi

In [138]:
for i, data_point in enumerate(labeled_data_dicts[1:]):
    label = data_point['label']
    text = data_point['text']
    initial_state = "Trial Start"
    state = initial_state if i == 0 else ordered_labels[i]
    response = chat_completion(state, text, max_tokens=250)
    print(f'Predicted Response: {response}')
    print(f'Ground Truth: {text, label}')
    print('-'*100)

Predicted Response: {"current_state": "Trial Start", "predicted_state": "Delay Start", "reason": "A fishing vessel is close to the trial area and could be a hazard to safe operations"}</s>
Ground Truth: ('Be aware, we do have a fishing vessel maybe 100 yards from our location.', 'Delay Start')
----------------------------------------------------------------------------------------------------
Predicted Response: {"current_state": Delay Start, "predicted_state": Delay End, "reason": "The text indicates that there are network problems which could delay the start of the trial."}</s>
Ground Truth: ("They're currently having network problems.", 'Delay Start')
----------------------------------------------------------------------------------------------------
Predicted Response: {"current_state": Delay Start, "predicted_state": Trial Start, "reason": "The text indicates that the trial will start at 1045."}</s>
Ground Truth: ("So 1015 for trial start, or I'm sorry, 1045 for trial start.", 'De